Note: This notebook assumes that "gmn_example1_extract_candidate_contrails.ipynb" has been run.

In this example, SAM2 is used to segment potential contrails in the images of extracted candidate flights.

In [1]:
import sys
sys.path.append("..")

from segmentation_utils import segment_contrails
from glob import glob
import time
import os
import torch

from sam2.build_sam import build_sam2_video_predictor

In [2]:
indiv_stations  = ['US0001']
all_stations = False
binary_threshold = 0  # SAM2 outputs logits rather than a binary mask. This can be tuned, 0 is the default binary threshold for SAM2
dates = [f'202308{str(x).zfill(2)}' for x in range(1, 30) if x != 27]
flight_id_filter = ['A03765_AAL300', 'A1463A_AAL2864', 'A0DADB_UPS855', 'A4B41C_AAL2328', 'ABB90D_SWA3676', 'A51B27_UPS853', 'A8C397_FDX1417', 'A28E5B_FDX1678', 'A22BF0_SWA279']

In [3]:
# Find all flights that have been extracted
flight_dirs = []

for date in dates:
    if all_stations:
        station_dirs = glob('../data/gmn_extracted_flight_images/*/')
        stations = []
        for station_dir in station_dirs:
            station_name = os.path.basename(os.path.normpath(station_dir))
            if os.path.isdir(os.path.join(station_dir, date)):
                stations.append(station_name)
            else:
                print(f'Could not find {station_name} for {date}, skipping')
    else:
        stations = indiv_stations

    for station in stations:
        curr_flight_dirs = [ f.path for f in os.scandir(f'../data/gmn_extracted_flight_images/{station}/{date}/') if f.is_dir() ]
        curr_flight_dirs = [x for x in curr_flight_dirs if os.path.basename(x) in flight_id_filter]

        if len(curr_flight_dirs) > 0:
            if len(curr_flight_dirs) < 10:
                print(f'Warning: found fewer than 10 flights for {station} on {date}, it is likely that the camera calibration is faulty due to cloudy weather')
            flight_dirs += curr_flight_dirs
        else:
            print(f'Could not find find flights for {station} on {date}, skipping')

Could not find find flights for US0001 on 20230801, skipping
Could not find find flights for US0001 on 20230802, skipping
Could not find find flights for US0001 on 20230804, skipping
Could not find find flights for US0001 on 20230805, skipping
Could not find find flights for US0001 on 20230806, skipping
Could not find find flights for US0001 on 20230807, skipping
Could not find find flights for US0001 on 20230808, skipping
Could not find find flights for US0001 on 20230809, skipping
Could not find find flights for US0001 on 20230810, skipping
Could not find find flights for US0001 on 20230811, skipping
Could not find find flights for US0001 on 20230812, skipping
Could not find find flights for US0001 on 20230813, skipping
Could not find find flights for US0001 on 20230814, skipping
Could not find find flights for US0001 on 20230817, skipping
Could not find find flights for US0001 on 20230818, skipping
Could not find find flights for US0001 on 20230819, skipping
Could not find find flig

In [4]:
# Only initialise the SAM2 predictor once rather than in the loop
# This solved a memory leak issue and also significantly speeds up the process
sam2_checkpoint="../sam2_checkpoints/sam2_hiera_large.pt"
sam2_cfg="sam2_hiera_l.yaml"
predictor = build_sam2_video_predictor(sam2_cfg, sam2_checkpoint)

In [6]:
# Run the SAM2 segmentation
for flight_dir in flight_dirs:
    flight_id = os.path.basename(flight_dir)

    with torch.no_grad():
        try:
            output_path = os.path.join(flight_dir, 'sam2_output')
            if os.path.exists(output_path) and len(glob(os.path.join(output_path, '*.png'))) >= 100:
                print(f"{flight_dir} already segmented")
                # continue
            elif glob(os.path.join(flight_dir, 'metadata', '*.json')) == []:
                print(f"{flight_dir} has no saved metadata, skipping")
                continue

            start_time = time.time()
            flight_id = os.path.basename(flight_dir)
            segment_contrails(predictor, flight_dir, debug=True, binary_threshold=binary_threshold)
            print(f"{flight_dir} took {time.time() - start_time} seconds")
        except Exception as e:
            print(f"Flight {flight_dir} Error: {e}")

../data/gmn_extracted_flight_images/US0001/20230803/A1463A_AAL2864 already segmented


frame loading (JPEG): 100%|██████████| 101/101 [00:03<00:00, 30.97it/s]
D:\Clouds\clean_contrails\segment-anything-2\sam2\modeling\backbones\hieradet.py:68: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(
D:\Clouds\clean_contrails\segment-anything-2\sam2\modeling\sam\transformer.py:270: UserWarning: Memory efficient kernel not used because: (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:718.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
D:\Clouds\clean_contrails\segment-anything-2\sam2\modeling\sam\transformer.py:270: UserWarning: Memory Efficient attention has been runtime disabled. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten

../data/gmn_extracted_flight_images/US0001/20230803/A1463A_AAL2864 took 16.260714769363403 seconds
../data/gmn_extracted_flight_images/US0001/20230815/A22BF0_SWA279 already segmented


propagate in video: 100%|██████████| 101/101 [00:09<00:00, 10.88it/s]


../data/gmn_extracted_flight_images/US0001/20230815/A22BF0_SWA279 took 14.901540517807007 seconds
../data/gmn_extracted_flight_images/US0001/20230815/A28E5B_FDX1678 already segmented


...

../data/gmn_extracted_flight_images/US0001/20230820/ABB90D_SWA3676 took 15.208412170410156 seconds
../data/gmn_extracted_flight_images/US0001/20230822/A8C397_FDX1417 already segmented


propagate in video: 100%|██████████| 101/101 [00:09<00:00, 10.60it/s]


../data/gmn_extracted_flight_images/US0001/20230822/A8C397_FDX1417 took 14.007265329360962 seconds
../data/gmn_extracted_flight_images/US0001/20230823/A03765_AAL300 already segmented


propagate in video: 100%|██████████| 101/101 [00:09<00:00, 10.49it/s]

../data/gmn_extracted_flight_images/US0001/20230823/A03765_AAL300 took 17.71648359298706 seconds
